In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="68TRJG3oL0xut9Ks0kkD")
project = rf.workspace("ship-xallx").project("test-au9yp")
version = project.version(5)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to test-5 in yolov8:: 100%|██████████| 11019/11019 [00:01<00:00, 6076.13it/s]


In [ ]:
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
import torch
from torch import nn
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from transformers import ViTForImageClassification, AdamW
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from collections import Counter
from PIL import Image
import os

In [ ]:
import os

# 디렉토리 경로 설정 (예시 경로)
base_path = "/content/test-5"

# 이미지 확장자 리스트
image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]

# 각 폴더의 이미지 파일 개수를 출력
def count_images_in_folder(folder_path):
    count = 0
    for root, _, files in os.walk(folder_path):
        count += len([file for file in files if os.path.splitext(file)[1].lower() in image_extensions])
    return count

folders = ["test/images", "train/images", "valid/images"]

# 결과 출력
for folder in folders:
    full_path = os.path.join(base_path, folder)
    if os.path.exists(full_path):
        print(f"{folder} 폴더의 이미지 개수: {count_images_in_folder(full_path)}개")
    else:
        print(f"{folder} 폴더가 존재하지 않습니다.")

test/images 폴더의 이미지 개수: 100개
train/images 폴더의 이미지 개수: 5204개
valid/images 폴더의 이미지 개수: 200개


In [ ]:
import os
from collections import Counter

# 기존 폴더 경로
base_path = "/content/test-5"
folders = ["train", "valid", "test"]

# 모든 파일 이름 수집
all_files = []
for folder in folders:
    images_path = os.path.join(base_path, folder, "images")
    if os.path.exists(images_path):
        all_files.extend(os.listdir(images_path))

# 파일 이름 중복 확인
duplicate_files = [file for file, count in Counter(all_files).items() if count > 1]
if duplicate_files:
    print("중복된 파일 이름:", duplicate_files)
else:
    print("중복된 파일 이름이 없습니다.")


중복된 파일 이름이 없습니다.


In [ ]:
import os
import shutil

# 기존 폴더 경로
combined_images_path = "/content/test-5/combined_images"
combined_labels_path = "/content/test-5/combined_labels"

# 폴더 초기화
if os.path.exists(combined_images_path):
    shutil.rmtree(combined_images_path)  # 폴더 전체 삭제
    os.makedirs(combined_images_path)   # 빈 폴더 다시 생성

if os.path.exists(combined_labels_path):
    shutil.rmtree(combined_labels_path)  # 폴더 전체 삭제
    os.makedirs(combined_labels_path)    # 빈 폴더 다시 생성

print("폴더 초기화 완료: combined_images와 combined_labels가 비워졌습니다.")


폴더 초기화 완료: combined_images와 combined_labels가 비워졌습니다.


In [ ]:
import os
import shutil

# 기존 폴더 경로
base_path = "/content/test-5"
folders = ["train", "valid", "test"]

# 합쳐질 임시 폴더 생성
combined_images_path = os.path.join(base_path, "combined_images")
combined_labels_path = os.path.join(base_path, "combined_labels")
os.makedirs(combined_images_path, exist_ok=True)
os.makedirs(combined_labels_path, exist_ok=True)

# 이미지 및 라벨 파일 합치기
for folder in folders:
    images_path = os.path.join(base_path, folder, "images")
    labels_path = os.path.join(base_path, folder, "labels")

    if os.path.exists(images_path):
        for image_file in os.listdir(images_path):
            # 이미지 파일 복사
            shutil.copy(os.path.join(images_path, image_file), combined_images_path)

            # 라벨 파일 매칭 및 복사
            label_file = os.path.splitext(image_file)[0] + ".txt"  # 확장자를 .txt로 변경
            if os.path.exists(os.path.join(labels_path, label_file)):
                shutil.copy(os.path.join(labels_path, label_file), combined_labels_path)

# 합친 데이터 확인
all_images = os.listdir(combined_images_path)
all_labels = os.listdir(combined_labels_path)

print(f"combined_images 폴더의 이미지 개수: {len(all_images)}개")
print(f"combined_labels 폴더의 라벨 개수: {len(all_labels)}개")


combined_images 폴더의 이미지 개수: 5504개
combined_labels 폴더의 라벨 개수: 5504개


In [ ]:
import os

# 폴더 경로
combined_images_path = "/content/test-5/combined_images"
combined_labels_path = "/content/test-5/combined_labels"

# 이미지와 라벨 파일 목록 가져오기
image_files = os.listdir(combined_images_path)
label_files = os.listdir(combined_labels_path)

# 이미지 이름에서 확장자를 제거한 리스트
image_base_names = [os.path.splitext(image_file)[0] for image_file in image_files]

# 라벨 이름에서 확장자를 제거한 리스트
label_base_names = [os.path.splitext(label_file)[0] for label_file in label_files]

# 매칭 확인
unmatched_images = [image for image in image_base_names if image not in label_base_names]
unmatched_labels = [label for label in label_base_names if label not in image_base_names]

# 결과 출력
if not unmatched_images and not unmatched_labels:
    print("모든 이미지와 라벨이 정확히 매칭되었습니다!")
else:
    if unmatched_images:
        print(f"매칭되지 않은 이미지 파일: {unmatched_images}")
    if unmatched_labels:
        print(f"매칭되지 않은 라벨 파일: {unmatched_labels}")


모든 이미지와 라벨이 정확히 매칭되었습니다!


In [ ]:
import os
import shutil
import random

# 기존 combined 데이터 경로
combined_images_path = "/content/test-5/combined_images"
combined_labels_path = "/content/test-5/combined_labels"

new_folders = {
    "train": {"images": "/content/new_data/train/images", "labels": "/content/new_data/train/labels"},
    "valid": {"images": "/content/new_data/valid/images", "labels": "/content/new_data/valid/labels"},
    "test": {"images": "/content/new_data/test/images", "labels": "/content/new_data/test/labels"},
}


for folder_type, paths in new_folders.items():
    os.makedirs(paths["images"], exist_ok=True)
    os.makedirs(paths["labels"], exist_ok=True)

# combined 데이터 리스트 가져오기
all_images = os.listdir(combined_images_path)
all_labels = os.listdir(combined_labels_path)

# 데이터 셔플 (랜덤 순서로 섞기)
random.seed(42)  # 재현성을 위한 고정 시드
combined_data = list(zip(all_images, all_labels))
random.shuffle(combined_data)

# 비율에 따라 분할
total_count = len(combined_data)
train_count = int(total_count * 0.6)
valid_count = int(total_count * 0.3)
test_count = total_count - train_count - valid_count

train_data = combined_data[:train_count]
valid_data = combined_data[train_count:train_count + valid_count]
test_data = combined_data[train_count + valid_count:]

# 분할된 데이터를 각각의 폴더로 이동
def move_data(data, folder_paths):
    for image_file, label_file in data:
        # 이미지와 라벨 파일 이동
        shutil.copy(os.path.join(combined_images_path, image_file), folder_paths["images"])
        shutil.copy(os.path.join(combined_labels_path, label_file), folder_paths["labels"])

move_data(train_data, new_folders["train"])
move_data(valid_data, new_folders["valid"])
move_data(test_data, new_folders["test"])

# 결과 출력
print(f"train 데이터: {len(train_data)}개")
print(f"valid 데이터: {len(valid_data)}개")
print(f"test 데이터: {len(test_data)}개")


train 데이터: 3302개
valid 데이터: 1651개
test 데이터: 551개


In [ ]:
# 사용자 정의 Dataset 클래스
class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = []
        self.labels = []

        for file in os.listdir(image_dir):
            if file.endswith(('jpg', 'jpeg', 'png')):
                label_file = file.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt')
                label_path = os.path.join(label_dir, label_file)
                if os.path.exists(label_path):
                    try:
                        label = self.extract_label(label_file)
                        self.image_files.append(file)
                        self.labels.append(label)
                    except KeyError:
                        print(f"Skipping file (unknown class): {file}")
                else:
                    print(f"Skipping file (missing label): {file}")

    def extract_label(self, label_file):
        class_name = label_file.split('_')[1]
        class_map = {'drift': 0, 'fish': 1, 'move': 2}
        if class_name in class_map:
            return class_map[class_name]
        else:
            raise KeyError(f"Unknown class name: {class_name}")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        label = self.labels[idx]
        with Image.open(img_path) as img:
            if self.transform:
                img = self.transform(img)
        return img, label

In [ ]:
base_path = '/content/new_data'

In [ ]:
# 데이터 경로 설정
train_image_dir = os.path.join(base_path, "train/images")
train_label_dir = os.path.join(base_path, "train/labels")

# 데이터 전처리
transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# 데이터 경로 설정
valid_image_dir = os.path.join(base_path, "valid/images")
valid_label_dir = os.path.join(base_path, "valid/labels")

In [ ]:
train_dataset = CustomDataset(train_image_dir, train_label_dir, transform=transform)

Skipping file (unknown class): 3-1-drift_jpg.rf.7461a9f8e800b1af0d23b52f5397eda5.jpg
Skipping file (missing label): 9-7_fish_jpg.rf.ec81f7ba02d6e941299a1b309c9e6033.jpg
Skipping file (missing label): 58-8_drift_jpg.rf.e23b31d121a9d37872056e5978f63f3e.jpg
Skipping file (missing label): 138-9_drift_jpg.rf.c4d08d68cadb8ae78ef208694bd5ce36.jpg
Skipping file (missing label): 22-8_move_jpg.rf.cc774d5dc7b6a1d14d90a3a40026270e.jpg
Skipping file (missing label): 16-6_move_jpg.rf.a0df20f38eacd64da93d00fa72c6d72f.jpg
Skipping file (unknown class): 43-2-move_jpg.rf.c55754d2605cd872a10d786c7090caf3.jpg
Skipping file (missing label): 40-7_move_jpg.rf.b64906027bbfb961f200a6eca0f45907.jpg
Skipping file (missing label): 65-8_drift_jpg.rf.65fb862d8145864fea63a7cfdbbbdfd8.jpg
Skipping file (missing label): 7-3-drift_jpg.rf.72363a17c12600f4379f0879c24f8eae.jpg
Skipping file (missing label): 24-3-drift_jpg.rf.4fbe85c71c07dbd3e366755c2222879d.jpg
Skipping file (missing label): 11-7_drift_jpg.rf.5283f8f0a973

In [ ]:
valid_dataset = CustomDataset(valid_image_dir, valid_label_dir, transform=transform)

Skipping file (missing label): 3-1-drift_jpg.rf.7461a9f8e800b1af0d23b52f5397eda5.jpg
Skipping file (missing label): 217-9_drift_jpg.rf.402c2433c8ebefe3c4960aa154c46dbd.jpg
Skipping file (missing label): 59-7_move_jpg.rf.17a0fce22b3c40cc357c86943457aefa.jpg
Skipping file (missing label): 268-9_drift_jpg.rf.8b1fe36c7ce587275a63df2fc8e76a98.jpg
Skipping file (missing label): 223-9_drift_jpg.rf.b065e5ffbcdb096d9cabc15c858f8a3c.jpg
Skipping file (missing label): 104-7_move_jpg.rf.0141c42cdaffa46a581476aba20bab56.jpg
Skipping file (missing label): 50-9_drift_jpg.rf.83efe3e061322dfdbd1710c10d78de84.jpg
Skipping file (missing label): 73-7_drift_jpg.rf.49a5664c0c90d4ac281973082f47b33f.jpg
Skipping file (missing label): 106-7_drift_jpg.rf.aa8e759800b8fc4e9b76a917fceb5c65.jpg
Skipping file (unknown class): 1-3-drift_jpg.rf.e56c60577bb6e47020d84ddd58130a34.jpg
Skipping file (missing label): 17-8_fish_jpg.rf.51431e79ef6cdfec2b19c6b1d7cfea1e.jpg
Skipping file (missing label): 49-1-fish_jpg.rf.4472c0

In [ ]:
# 클래스 분포 확인 및 가중치 계산
train_labels = [label for _, label in train_dataset]
label_counts = Counter(train_labels)
total_samples = sum(label_counts.values())
class_weights = [total_samples / label_counts[i] for i in range(len(label_counts))]
class_weights = torch.tensor(class_weights).to('cuda' if torch.cuda.is_available() else 'cpu')

print("Class distribution:", label_counts)
print("Class weights:", class_weights)

Class distribution: Counter({0: 1097, 2: 837, 1: 402})
Class weights: tensor([2.1294, 5.8109, 2.7909], device='cuda:0')


In [ ]:
# DataLoader 생성
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

# 모델 생성
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224-in21k", num_labels=3)
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# 손실 함수 및 옵티마이저
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = AdamW(model.parameters(), lr=1e-4)

from torch.optim.lr_scheduler import ReduceLROnPlateau
# 학습률 스케줄러
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning


In [ ]:
# 학습 루프
def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, epochs=20, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(pixel_values=images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # 검증 단계
        model.eval()
        valid_loss = 0.0
        valid_labels = []
        valid_preds = []
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(pixel_values=images).logits
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                preds = torch.argmax(outputs, dim=1)
                valid_labels.extend(labels.cpu().numpy())
                valid_preds.extend(preds.cpu().numpy())

        accuracy = accuracy_score(valid_labels, valid_preds)
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Valid Loss: {valid_loss/len(valid_loader):.4f}, Accuracy: {accuracy:.2f}")

        # 학습률 스케줄러 업데이트
        scheduler.step(valid_loss)

        # Early Stopping
        if valid_loss < best_loss:
            best_loss = valid_loss
            patience_counter = 0
            torch.save(model.state_dict(), "best_vit_model.pth")  # 최적 모델 저장
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered!")
                break

In [ ]:
# 모델 학습
train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, epochs=12, patience=4)

Training Epoch 1/12: 100%|██████████| 146/146 [01:37<00:00,  1.50it/s]


Epoch 1/12, Train Loss: 0.9447, Valid Loss: 0.8557, Accuracy: 0.69


Training Epoch 2/12: 100%|██████████| 146/146 [01:39<00:00,  1.46it/s]


Epoch 2/12, Train Loss: 0.5917, Valid Loss: 0.5396, Accuracy: 0.81


Training Epoch 3/12: 100%|██████████| 146/146 [01:40<00:00,  1.46it/s]


...

Epoch 10/12, Train Loss: 0.0050, Valid Loss: 0.3155, Accuracy: 0.93


Training Epoch 11/12: 100%|██████████| 146/146 [01:40<00:00,  1.46it/s]


Epoch 11/12, Train Loss: 0.0034, Valid Loss: 0.3231, Accuracy: 0.93


Training Epoch 12/12: 100%|██████████| 146/146 [01:40<00:00,  1.46it/s]


Epoch 12/12, Train Loss: 0.0027, Valid Loss: 0.3295, Accuracy: 0.93


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# 테스트 데이터 로드
test_image_dir = os.path.join(base_path, "test/images")
test_label_dir = os.path.join(base_path, "test/labels")

test_dataset = CustomDataset(test_image_dir, test_label_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# 테스트 평가 함수
def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()  # 평가 모드로 전환

    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(pixel_values=images).logits
            predictions = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    # 성능 평가
    accuracy = accuracy_score(all_labels, all_predictions)
    report = classification_report(all_labels, all_predictions, target_names=['drift', 'fish', 'move'], digits=4)

    print("=== 테스트 데이터 평가 결과 ===")
    print(f"Accuracy: {accuracy:.4f}")
    print("\nClassification Report:\n", report)

Skipping file (missing label): 120-7_drift_jpg.rf.140a631832333f951932c29a0c3d6717.jpg
Skipping file (missing label): 56-9_move_jpg.rf.e3c8ba66f1d42e14436db6031af81430.jpg
Skipping file (missing label): 13-2-fish_jpg.rf.bc005249ffa952585ee7d0a4ac5d8ed5.jpg
Skipping file (missing label): 178-9_drift_jpg.rf.44abdafd15665b189d2c5fd912f21e88.jpg
Skipping file (missing label): 291-9_drift_jpg.rf.095c998a350e6e66efc2cae81c8bef21.jpg
Skipping file (missing label): 15-10_move_jpg.rf.3836ad15d8f0040cbe5d6ecda630591d.jpg
Skipping file (missing label): 13-8_move_jpg.rf.a092b2a4f7c3378d62f532ec2a3d4445.jpg
Skipping file (missing label): 24-7_fish_jpg.rf.d9e3d8e4627d5cac328a81a27f45ecec.jpg
Skipping file (missing label): 97-7_fish_jpg.rf.f924ff1a26b75dc5f1b11fde829a2527.jpg
Skipping file (missing label): 6-1-move_jpg.rf.c5509ed1d41b8227ce70574c128beffd.jpg
Skipping file (missing label): 300-9_drift_jpg.rf.13cb9de44de4df6f31a2fb97803280cf.jpg
Skipping file (missing label): 77-9_move_jpg.rf.ae4fc6b76

In [ ]:

# 저장된 모델 로드
model.load_state_dict(torch.load("best_vit_model.pth"))

# 테스트 평가 실행
evaluate_model(model, test_loader)

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


=== 테스트 데이터 평가 결과 ===
Accuracy: 0.9074

Classification Report:
               precision    recall  f1-score   support

       drift     0.8462    0.9565    0.8980        23
        fish     0.8889    0.8889    0.8889         9
        move     1.0000    0.8636    0.9268        22

    accuracy                         0.9074        54
   macro avg     0.9117    0.9030    0.9046        54
weighted avg     0.9160    0.9074    0.9082        54

